In [414]:
import re
import simdjson as sj
import gzip
from tqdm import tqdm
import pandas as pd
from tqdm.notebook import tqdm
import swifter
import seaborn as sns
import numpy as np	
from nltk import sent_tokenize
import re
from sentence_transformers import SentenceTransformer
from wutils.general import save_pickle, load_pickle
from wutils.mat import MarkedMatrix
from scipy.spatial.distance import cdist
from sklearn.linear_model import LogisticRegression

In [415]:
cord_df = load_pickle('./pickles/who_cord_df.pkl')

In [ ]:
mm_df = load_pickle('../who')

In [416]:
# cord_df[cord_df.abstract.swifter.apply(lambda x: 'shortages' in x)]

In [417]:
good_df = load_pickle('./pickles/newest_good_sample_df_ft.pkl')
bad_df = load_pickle('./pickles/newest_bad_df_ft.pkl')

In [418]:
import urllib.parse
def get_hostname(url):
    o = urllib.parse.urlsplit(url)
    return o.hostname

In [419]:
good_df['hostname'] = good_df['url'].swifter.apply(get_hostname)
bad_df['hostname'] = bad_df['url'].swifter.apply(get_hostname)

In [420]:
good_df.hostname.value_counts()

www.reuters.com                 5630
www.thehindu.com                5158
www.france24.com                3497
www.forbes.com                  3433
indianexpress.com               3188
                                ... 
www.spokesman.com                  1
www.mediamatters.org               1
verysmartbrothas.theroot.com       1
newsinteractives.cbc.ca            1
obits.lohud.com                    1
Name: hostname, Length: 238, dtype: int64

In [421]:
good_df.hostname.value_counts()

www.reuters.com                 5630
www.thehindu.com                5158
www.france24.com                3497
www.forbes.com                  3433
indianexpress.com               3188
                                ... 
www.spokesman.com                  1
www.mediamatters.org               1
verysmartbrothas.theroot.com       1
newsinteractives.cbc.ca            1
obits.lohud.com                    1
Name: hostname, Length: 238, dtype: int64

In [9]:
# target_df = bad_df[bad_df.hostname == 'beforeitsnews.com']
# target_df = target_df[target_df.covidFreq > 20]

In [422]:
target_df = good_df[good_df.hostname == 'www.reuters.com']

In [95]:
target_df.sentences.iloc[505]

['Feb 14 () - The U.S. State Department will evacuate some American citizens and their families aboard the Diamond Princess cruise ship, the biggest cluster of coronavirus infections outside China, the Wall Street Journal reported on Friday.',
 'The Diamond Princess cruise ship, owned by Carnival Corp , has been quarantined since arriving in Yokohama, Japan, on Feb. 3, with 3,500 passengers and crew on board, of whom 218 have tested positive for the coronavirus infection.',
 'Those who have tested positive for the virus have been taken off the ship to hospital.',
 '(Reporting by Aakriti Bhalla in Bengaluru and Kiyoshi Takenaka in Tokyo Editing by Leslie Adler)']

In [12]:
cord_df

,cord_uid,source_x,title,abstract,authors,sentences,sent_embeddings,ft_embeddings
104326,2g4o1alu,WHO,COVID-19-related mortality by age groups in Eu...,"Background and Objectives: To date, more than ...","Cohen, JÃ©rÃ©mie F.; Korevaar, DaniÃ«l A.; Mat...",[COVID-19-related mortality by age groups in E...,"[[-0.055676993, -0.9243626, 0.6984887, -1.5472...","[[0.0481136, -0.045153778, -0.0016712277, -0.0..."
104355,bbumotlt,WHO,Inhibitors for Novel Coronavirus Protease Iden...,The rapid outbreak of the novel severe acute r...,"Fischer, André; Sellner, Manuel; Neranjan, San...",[Inhibitors for Novel Coronavirus Protease Ide...,"[[-0.23659688, -1.209985, 1.086681, -0.2994868...","[[0.019563006, 0.0033947488, 0.04383449, -0.06..."
104378,fmfp5a4x,WHO,Induction of “cytokine storm” in patients infe...,Coronaviruses are an extensive family of virus...,"Parra-Izquierdo, V.; Flórez-Sarmiento, C.; Rom...",[Induction of “cytokine storm” in patients inf...,"[[-0.6050015, -1.024411, 0.5046835, -0.9459065...","[[0.0046710027, -0.07187022, 0.008835351, -0.0..."
104382,n8muqsk8,WHO,The case series of 34 patients with COVID-19 d...,Background: A novel coronavirus (SARS-CoV-2) c...,"Kase, Kerstin; Skrzat-Klapaczynska, Agata; Vas...",[The case series of 34 patients with COVID-19 ...,"[[-0.5378597, -1.2903348, 0.29853648, -0.45812...","[[0.00923657, -0.010481103, 0.012520884, -0.03..."
104388,2rlld533,WHO,A brief overview of current drug repurposing a...,This brief overview is intended to shed light ...,"Fonseca Soares Rodrigues, P. da Thomaz D. V.",[A brief overview of current drug repurposing ...,"[[-0.84153324, -0.8163958, 0.36981186, -0.9351...","[[0.010491839, -0.066567436, -0.018717863, -0...."
...,...,...,...,...,...,...,...,...
301152,mbv6ot95,WHO,Statistical evaluation of selected air quality...,Air pollution has become a serious concern for...,"Masum, M. H.; Pal, S. K.",[Statistical evaluation of selected air qualit...,"[[-0.6117112, -0.66956645, 0.6793621, -0.59324...","[[-0.022183394, 0.018007744, 0.026416264, -0.0..."
311269,jfoflgah,WHO,Clinical features of patients with probable 20...,BACKGROUND: Coronavirus disease-19 COVID-19 is...,"Karkhah, S.; Ghazanfari, M. J.; Shamshirian, A...",[Clinical features of patients with probable 2...,"[[-0.22488534, -0.8938913, 0.7023613, -1.39347...","[[-0.010006628, -0.07970685, -0.0008492946, -0..."
312759,e4pr78n0,WHO,Mathematical modeling of COVID-19 transmission...,BackgroundWe evaluated how non-pharmaceutical ...,"Tuite, Ashleigh; Fisman, David N; Greer, Amy L",[Mathematical modeling of COVID-19 transmissio...,"[[-0.9591663, -0.7535791, 0.11427775, -0.55517...","[[-0.0031259845, -0.045985896, 0.01022184, -0...."
331600,j90w6so2,WHO,Impact of COVID-19 large scale restriction on ...,"COVID-19 has a severe and widespread impact, e...","Caraka, R. E. Lee Y. Kurniawan R. Herliansyah ...",[Impact of COVID-19 large scale restriction on...,"[[-0.6988445, -0.4860237, -0.33028847, -0.6850...","[[-0.022731125, -0.008827577, 0.009168559, -0...."


In [381]:
with open('./data/words.txt') as f:
    words = f.readlines()

In [392]:
words = [x.strip().lower() for x in words]

In [393]:
import random
sents = []
for i in range(5000):
    n_words = random.randint(10, 30)
    sent_words = random.choices(words, k=n_words)
    sent = ' '.join(sent_words)
    sent = sent[0].upper() + sent[1:] + '.'
    sents.append(sent)

In [396]:
sent_encoder = SentenceTransformer('distilbert-base-nli-mean-tokens')

I1205 04:09:12.194085 139722638161728 SentenceTransformer.py:38] Load pretrained SentenceTransformer: distilbert-base-nli-mean-tokens
I1205 04:09:12.195493 139722638161728 SentenceTransformer.py:42] Did not find folder distilbert-base-nli-mean-tokens
I1205 04:09:12.196023 139722638161728 SentenceTransformer.py:48] Try to download model from server: https://sbert.net/models/distilbert-base-nli-mean-tokens.zip
I1205 04:09:12.197632 139722638161728 SentenceTransformer.py:99] Load SentenceTransformer from folder: /home/will/.cache/torch/sentence_transformers/sbert.net_models_distilbert-base-nli-mean-tokens
I1205 04:09:13.403727 139722638161728 SentenceTransformer.py:123] Use pytorch device: cuda


In [397]:
encoded = sent_encoder.encode(sents, convert_to_tensor=True, show_progress_bar=True)

In [399]:
encoded = encoded.numpy()

In [394]:
# true_df['text'].swifter.apply(lambda x: 'myth' in x.lower()).sum()

In [423]:
KW = 'vaccine'
print(good_df['cleanText'].swifter.apply(lambda x: KW in x.lower()).sum())
print(bad_df['cleanText'].swifter.apply(lambda x: KW in x.lower()).sum())


7925



2876


In [323]:
# vax_good_df = good_df.sample(n=20000).dropna()

In [324]:
# vax_bad_df = bad_df.sample(n=20000).dropna()

In [434]:
vax_good_df = good_df[good_df['cleanText'].swifter.apply(lambda x: KW in x.lower())]
vax_bad_df = bad_df[bad_df['cleanText'].swifter.apply(lambda x: KW in x.lower())]

In [435]:
vax_good_df['mean_embed'] = vax_good_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))
vax_bad_df['mean_embed'] = vax_bad_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))
# vax_good_df['mean_embed'] = vax_good_df['ft_embeddings'].swifter.apply(lambda x: np.array([y for y in x if np.isfinite(y).all()]).mean(axis=0))
# vax_bad_df['mean_embed'] = vax_bad_df['ft_embeddings'].swifter.apply(lambda x: np.array([y for y in x if np.isfinite(y).all()]).mean(axis=0))

/home/will/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


/home/will/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [436]:
vax_good_mat = np.vstack(vax_good_df.mean_embed)
vax_bad_mat = np.vstack(vax_bad_df.mean_embed)

In [437]:
vax_mm = MarkedMatrix([('good', vax_good_mat), ('bad', vax_bad_mat)])

In [427]:
np.where(~np.isfinite(vax_mm.mat))

(array([], dtype=int64), array([], dtype=int64))

In [438]:
vax_mm.single_split_classify(model)

Training model...
Evaluating model...


/home/will/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


(0.8374676045908923, 0.8945472015373528)

In [103]:
model = LogisticRegression(max_iter=400)

In [439]:
cord_df = cord_df.dropna()

In [440]:
has_vax = cord_df[cord_df.title.swifter.apply(lambda x: KW in x.lower()) | cord_df.abstract.swifter.apply(lambda x: KW in x.lower())]
# has_vax = cord_df.sample(n=5000)

In [441]:
has_vax['mean_embed'] = has_vax['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))
# has_vax['mean_embed'] = has_vax['ft_embeddings'].swifter.apply(lambda x: np.array(x).mean(axis=0))

/home/will/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [442]:
vax_ref = np.vstack(has_vax['mean_embed'])

In [443]:
vax_ref.shape

(4686, 768)

In [444]:
vax_good_mat.shape

(7925, 768)

In [ ]:
# vgood2ref = cdist(vax_good_mat, vax_ref, 'cosine')

In [405]:
# vgood2ref = cdist(vax_good_mat, encoded, 'cosine')
# print('done w/ good')
# vbad2ref = cdist(vax_bad_mat, encoded, 'cosine')

done w/ good


In [445]:
vgood2ref = cdist(vax_good_mat, vax_ref, 'cosine')
print('done w/ good')
vbad2ref = cdist(vax_bad_mat, vax_ref, 'cosine')

done w/ good


In [377]:
vax_ref[~np.isfinite(vax_ref)] = 0

In [446]:
vax_mm = MarkedMatrix([('good', vgood2ref), ('bad', vbad2ref)])

In [458]:
vax_raw_mm = MarkedMatrix([('good', vax_good_mat), ('bad', vax_bad_mat)])

In [447]:
claims_df = load_pickle('./pickles/newest_claims_df_labelled.pkl')
claims_df = claims_df.dropna()

true_claims_df = claims_df[claims_df['label'] == 'true']
false_claims_df = claims_df[claims_df['label'] == 'false']
misleading_claims_df = claims_df[claims_df['label'] == 'misleading']

true_claims_mat = np.vstack(list(true_claims_df['bert_embedding']))
false_claims_mat = np.vstack(list(false_claims_df['bert_embedding']))

In [448]:
true_claims2ref = cdist(true_claims_mat, vax_ref, 'cosine')
false_claims2ref = cdist(false_claims_mat, vax_ref, 'cosine')

In [265]:
true_claims2ref = -cdist(true_claims_mat, vax_ref)
false_claims2ref = -cdist(false_claims_mat, vax_ref)

In [449]:
true_claims2ref.shape

(51, 4686)

In [450]:
claims_mm = MarkedMatrix([('true', true_claims_mat), ('false', false_claims_mat)])
claims_mm2covid = MarkedMatrix([('true', true_claims2ref), ('false', false_claims2ref)])

In [451]:
claims_mm.single_split_classify(tmodel, return_labels=True)

Training model...
Evaluating model...


((0.9296875, 0.9635627530364372), {'true': 0, 'false': 1})

In [278]:
tmodel_claims = RandomForestClassifier(n_estimators=500, n_jobs=24)

In [298]:
claims_mm2covid.single_split_classify(tmodel_claims, return_labels=True)

Training model...
Evaluating model...


((0.90625, 0.9504132231404958), {'true': 0, 'false': 1})

In [282]:
def tree_claim_explain_bad(idx):
    pred, bias, contrib = ti.predict(tmodel_claims, false_claims2ref[idx, :].reshape((1, -1)))
    print('CLEAN TEXT: ', false_claims_df.iloc[idx].text)
    cmat = contrib.reshape((-1, 2))
    print('Prediction: ', pred)
    print('Bias: ', bias)
    import_good = np.argsort(cmat[:, 0])[::-1]
    import_bad = np.argsort(cmat[:, 1])[::-1]
    print('===== Towards Bad: =====')
    for idx in import_bad[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')
    print('===== Towards Good: =====')
    for idx in import_good[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')

def tree_claim_explain_good(idx):
    pred, bias, contrib = ti.predict(tmodel_claims, true_claims2ref[idx, :].reshape((1, -1)))
    print('CLEAN TEXT: ', true_claims_df.iloc[idx].text)
    cmat = contrib.reshape((-1, 2))
    print('Prediction: ', pred)
    print('Bias: ', bias)
    import_good = np.argsort(cmat[:, 0])[::-1]
    import_bad = np.argsort(cmat[:, 1])[::-1]
    print('===== Towards Good: =====')
    for idx in import_good[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
#         print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')
    print('===== Towards Bad: =====')
    for idx in import_bad[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
#         print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')

In [293]:
tree_claim_explain_good(10)

CLEAN TEXT:  Schools have resumed in New Brunswick despite the COVID-19 pandemic
Prediction:  [[0.636 0.364]]
Bias:  [[0.10461619 0.89538381]]
===== Towards Good: =====
---------------------------------------------
CONTRIB: [ 0.02290918 -0.02290918]
TITLE: Towards reduction in bias in epidemic curves due to outcome misclassification through Bayesian analysis of time-series of laboratory test results: case study of COVID-19 in Alberta, Canada and Philadelphia, USA
---------------------------------------------
CONTRIB: [ 0.01990626 -0.01990626]
TITLE: Early spring near-surface ozone in Europe during the COVID-19 shutdown: Meteorological effects outweigh emission changes
---------------------------------------------
CONTRIB: [ 0.01544806 -0.01544806]
TITLE: The Successes and Failures of the Initial COVID-19 Pandemic Response in Romania
---------------------------------------------
CONTRIB: [ 0.01445399 -0.01445399]
TITLE: After the Hurricane: Economic Adversity, Bank Offices, and Communit

In [231]:
tree_claim_explain_bad(11)

CLEAN TEXT:  COVID-19 testing patent was submitted in 2015 and COVID19 is a scam
Prediction:  [[0.018 0.982]]
Bias:  [[0.10208355 0.89791645]]
===== Towards Bad: =====
---------------------------------------------
CONTRIB: [-0.00167625  0.00167625]
ABSTRACT: On March 11, 2020, COVID-19 was declared as a pandemic by World Health Organization. Criteria for identifying persons under investigation for SARS-CoV-2 infection by Centers for Disease Control and Prevention remain focusing on fever and respiratory symptoms. We report a case of COVID-19 patient who presented with colitis alone.
TITLE: Colitis as a Sole Presentation of SARS-CoV-2 Infection: Case Report
---------------------------------------------
CONTRIB: [-0.00121562  0.00121562]
ABSTRACT: This study quantifies the economic effect of a possible lockdown of Tokyo to prevent the spread of COVID-19. The negative effect of such a lockdown may propagate to other regions through supply chains because of supply and demand shortages. App

In [114]:
from sklearn.ensemble import RandomForestClassifier

In [357]:
from sklearn.tree import DecisionTreeClassifier
# tmodel = DecisionTreeClassifier()
tmodel = RandomForestClassifier(n_estimators=500, n_jobs=24)


In [362]:
vax_mm.mat[~np.isfinite(vax_mm.mat)] = 0

In [452]:
vax_mm.single_split_classify(tmodel, return_labels=True)

Training model...
Evaluating model...


((0.8056275453535727, 0.8787248787248788), {'bad': 0, 'good': 1})

In [407]:
vax_mm.single_split_classify(tmodel, return_labels=True)

Training model...
Evaluating model...


((0.7102, 0.7114695340501792), {'bad': 0, 'good': 1})

In [459]:
tmodel2 = RandomForestClassifier(n_estimators=500, n_jobs=24)

In [460]:
vax_raw_mm.single_split_classify(tmodel2, return_labels=True)

Training model...
Evaluating model...


((0.8267308404294705, 0.8927589367552704), {'bad': 0, 'good': 1})

In [461]:
def tree_explain_good(idx):
    pred, bias, contrib = ti.predict(tmodel, vgood2ref[idx, :].reshape((1, -1)))
    print('CLEAN TEXT: ', vax_good_df.iloc[idx].cleanText)
    cmat = contrib.reshape((-1, 2))
    print('Prediction: ', pred)
    print('Bias: ', bias)
    import_bad = np.argsort(cmat[:, 0])[::-1]
    import_good = np.argsort(cmat[:, 1])[::-1]
    print('===== Towards Good: =====')
    for idx in import_good[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')
    print('===== Towards Bad: =====')
    for idx in import_bad[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')

def tree_explain_bad(idx):
    pred, bias, contrib = ti.predict(tmodel, vbad2ref[idx, :].reshape((1, -1)))
    print('CLEAN TEXT: ', vax_bad_df.iloc[idx].cleanText)
    cmat = contrib.reshape((-1, 2))
    print('Prediction: ', pred)
    print('Bias: ', bias)
    import_bad = np.argsort(cmat[:, 0])[::-1]
    import_good = np.argsort(cmat[:, 1])[::-1]
    print('===== Towards Bad: =====')
    for idx in import_bad[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')
    print('===== Towards Good: =====')
    for idx in import_good[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')

In [457]:
tree_explain_bad(1917)

CLEAN TEXT:  (Natural News) The guy who’s credited with having first discovered the AIDS virus, and who won a Nobel Prize in Medicine for it, has made some unsettling claims about the alleged nature of the Wuhan coronavirus (COVID-19).
According to French virologist Luc Montagnier, the Wuhan coronavirus (COVID-19) didn’t originate at a Wuhan wet market like the mainstream media initially claimed. It actually came from a Wuhan laboratory, he contends, and contains the same DNA as HIV.
During an interview with Why Doctor? that took place on April 16, Montagnier explained that while it might be a “beautiful legend” that the Wuhan coronavirus came from tainted bat meat, the truth is that it’s actually a genetically modified (GMO) virus that had the HIV gene sequence artificially inserted into its genome.
“The Wuhan city laboratory has specialized in these coronaviruses since the early 2000s,” Montagnier stated, adding that “they have expertise in this area.” Along with his mathematician co

In [524]:
vax_good_df.iloc[157].cleanText

'More than 20m Americans lost their jobs in April – and Friday’s report suggests there might be much more trouble ahead\nFour causes for alarm in the US jobs figures – and one possible reason for hope\nFriday was a dark day for the US economy. The labor department announced more than 20 million people lost their jobs in April as the coronavirus shut down much of the economy.\nHere are five key takeaways from a report that will enter the history books as the worst since the Great Depression of the 1930s.\n1) The headline figures are bad – but what lies beneath is worse\nAbout 20.5m jobs were lost in the month and the unemployment rate shot up to 14.7%. The numbers are probably deflated, since a huge chunk of people have considered themselves out of the workforce, so they are not counted in the unemployment numbers.\nAbout 6.5 million more people filed for unemployment since the mid-April reference week Friday’s numbers are based on.\nAbout 6 million people left the workforce over the mo

In [526]:
tree_explain_good(157)

CLEAN TEXT:  More than 20m Americans lost their jobs in April – and Friday’s report suggests there might be much more trouble ahead
Four causes for alarm in the US jobs figures – and one possible reason for hope
Friday was a dark day for the US economy. The labor department announced more than 20 million people lost their jobs in April as the coronavirus shut down much of the economy.
Here are five key takeaways from a report that will enter the history books as the worst since the Great Depression of the 1930s.
1) The headline figures are bad – but what lies beneath is worse
About 20.5m jobs were lost in the month and the unemployment rate shot up to 14.7%. The numbers are probably deflated, since a huge chunk of people have considered themselves out of the workforce, so they are not counted in the unemployment numbers.
About 6.5 million more people filed for unemployment since the mid-April reference week Friday’s numbers are based on.
About 6 million people left the workforce over t

In [468]:
tree_explain_good(103)

CLEAN TEXT:  () - The new coronavirus is likely spreading through the air to some degree, the top U.S. infectious disease official said on Friday, one day after the World Health Organization urged further studies on the ways the virus is transmitted.
“Still some question about aerosol but likely some degree of aerosol,” Anthony Fauci, the head of the U.S. National Institute of Allergy and Infectious Diseases, said by video during a panel session at a COVID-19 conference organized by the International AIDS Society.
Fauci on Thursday had said it was a “reasonable assumption” that airborne transmission was occurring even though there was not a lot of solid evidence behind it. The WHO urged more studies on the issue.
The number of cases in the United States has surged to more than 3 million as states that quickly reopened bars and restaurants have reported sharp increases.
Fauci said that to battle the global virus, there needed to be multiple vaccine candidates. Vaccine makers are aiming 

In [35]:
from treeinterpreter import treeinterpreter as ti

In [37]:
pred, bias, contrib = ti.predict(tmodel, vbad2ref[290, :].reshape((1, -1)))
# pred, bias, contrib = ti.predict(tmodel, vbad2ref[290, :])

In [52]:
cmat = contrib.reshape((-1, 2))
import_bad = np.argsort(cmat[:, 0])[::-1]
import_good = np.argsort(cmat[:, 1])[::-1]

In [53]:
print(import_bad[:5])

[1398 1550 3095 3382 2875]


In [54]:
print(import_good[:5])

[1141  821 2810 2357 1929]


In [59]:
has_vax.iloc[2810].abstract

'Purpose: The goal of these recommendations is to provide guidance on the optimal care of children with glomerular diseases during the COVID-19 pandemic Patients with glomerular diseases are known to be more susceptible to infection Risk factors include decreased vaccine uptake, urinary loss of immunoglobulins, and treatment with immunosuppressive medications The Canadian Society of Nephrology (CSN) recently published guidelines on the care of adult glomerulonephritis patients This guideline aims to expand and adapt those recommendations for programs caring for children with glomerular diseases Sources of information: We used the CSN COVID-19 Rapid Response Team adult glomerulonephritis recommendations, published in the Canadian Journal of Kidney Health and Disease, as the foundation for our guidelines We reviewed documents published by nephrology and non-nephrology societies and health care agencies focused on kidney disease and immunocompromised populations Finally, we conducted a fo

In [ ]:
for k in range(contrib.shape[0]):
    for c, feature in zip(contrib[k], [f'doc{x}' for x in range(4686)]):
        if abs(c[0] - c[1]) < 0.000001:
            continue
        print(feature, c, c[0] - c[1])

In [38]:
contrib[0].shape

(1, 4686, 2)

In [ ]:
vax_mm.mat.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print("def tree({}):\n  path = []".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print("{}if {} <= {}:".format(indent, name, threshold))
            print("{}  print('{} <= {}')".format(indent, name, threshold))
            print("{}  path.append(({}, '<=', {}))".format(indent, name[3:], threshold))
            recurse(tree_.children_left[node], depth + 1)
            print("{}else:  # if {} > {}".format(indent, name, threshold))
            print("{}  print('{} > {}')".format(indent, name, threshold))
            print("{}  path.append(({}, '>', {}))".format(indent, name[3:], threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
#             print(tree_.value[node].shape)
            print("{}return ({}, path)".format(indent, list(tree_.value[node].ravel())))

    recurse(0, 1)


In [ ]:
def explain_tree(idx):
    (res, path) = tree(*vbad2ref[idx, :])
    print('RESULT: {}, {}'.format(res, tmodel.predict(vbad2ref[idx, :].reshape((1, -1)))))
    print('Clean Text: {}'.format(vax_bad_df.iloc[idx].cleanText))
    for term in path:
        name, op, thresh = term
        print('-'*25)
        print('CRITERIA: ({}): {} {}'.format(name, op, thresh))
        print('TITLE: {}'.format(has_vax.iloc[name].title))
        print('ABSTRACT: {}'.format(has_vax.iloc[name].abstract))

In [ ]:
explain_tree(310)

In [ ]:
tree_to_code(tmodel, [f'doc{x}' for x in range(4686)])

In [ ]:
tree(*vgood2ref[254, :])

In [ ]:
tree(*vgood2ref[255, :])

In [ ]:
vax_good_df.iloc[255].cleanText

In [ ]:
has_vax.iloc[4460].abstract

In [ ]:
plot_tree(tmodel)

In [ ]:
s_cord_df = cord_df.sample(n=3000)
good_mat = 

In [ ]:
def get_results(num, k=5):
    res = model.predict(vgood2ref[num, :].reshape(1, -1))
    print(res)
    weights = model.coef_ * vgood2ref[num, :].T
#     top_k = np.argsort(weights)[0, :k]
    top_k = np.argsort(weights)[0, :k]
    print('Clean Text: ', vax_good_df.iloc[num].cleanText)
    for el in top_k:
        print('-'*25)
        print(f'Weight: {weights[0, el]}')
        print(f'Related abstract ({el}): {has_vax.iloc[el].abstract}')
        print(f'Related title ({el}): {has_vax.iloc[el].title}')

In [ ]:
# get_results(2912)

In [ ]:
get_results(129)

In [ ]:
[(model.coef_[0, x], has_vax.iloc[x].abstract) for x in np.argsort(model.coef_[0, :])[::-1][:10]]

In [ ]:
cord_df

In [ ]:
vax_bad_df.iloc[0].mean_embed.shape

In [ ]:
false_df['text'].swifter.apply(lambda x: 'myth' in x.lower()).sum()

In [ ]:
false_df = claims_df[claims_df['label'] == 'false']
true_df = claims_df[claims_df['label'] == 'true']

In [ ]:
s_mat.iloc[0].shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from scipy.spatial.distance import cdist

In [ ]:
true_mat = np.vstack(true_df.bert_embedding)
fake_mat = np.vstack(false_df.bert_embedding)

In [ ]:
best_model = None
best_f1 = 0

for i in tqdm(range(1000)):
    s_cord = cord_df.sample(100)
    s_mat = np.vstack(s_cord['sent_embeddings'].swifter.progress_bar(False).apply(lambda x: x.mean(axis=0)))
    model = LogisticRegression(max_iter=1000)
    true2s = cdist(true_mat, s_mat, 'cosine') # try cosine?
    fake2s = cdist(fake_mat, s_mat, 'cosine')
    mm = MarkedMatrix([('true', true2s), ('fake', fake2s)])
    acc, f1 = mm.single_split_classify(model, verbose=False)
    if f1 > best_f1:
        best_f1 = f1
        best_model = model
        print('New best F1: ', best_f1)

In [ ]:
best_f1

In [ ]:
cnt = 0
for x in np.argsort(model.coef_[0, :])[::-1]:
    print(s_cord.iloc[x].title)
    cnt += 1
    if cnt > 1000:
        break

In [ ]:
claim_df = load_pickle('./pickles/newest_claims_df_labelled.pkl')

In [ ]:
claim_df

In [ ]:
claims_mat = np.array(list(claim_df['bert_embedding']))

In [ ]:
target_df.iloc[11].cleanText

In [ ]:
bad_df[bad_df.hostname == 'beforeitsnews.com'].iloc[4].cleanText

In [ ]:
mm_df = load_pickle('./pickles/mm_df.pkl').dropna()

In [ ]:
cord_df

In [ ]:
cord_df['mean_embed'] = cord_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))

In [ ]:
s_cord = cord_df.sample(n=100)

In [ ]:
has_vax = cord_df['abstract'].swifter.apply(lambda x: 'vaccine' in x)

In [ ]:
s_cord = cord_df[has_vax]

In [ ]:
mat_cord = np.vstack(s_cord['mean_embed'])

In [ ]:
real_df = mm_df[mm_df['label'] == 'real']
fake_df = mm_df[mm_df['label'] == 'fake']

In [ ]:
fake_df = fake_df[fake_df.text != 'NONE']

In [ ]:
good_mat = np.vstack(real_df['mean_embed'])
bad_mat = np.vstack(mm_df[mm_df['label'] == 'fake']['mean_embed'])

In [ ]:
mm = MarkedMatrix((('real', good_mat), ('fake', bad_mat)))

In [ ]:
good2cord = cdist(good_mat, mat_cord, 'cosine')
bad2cord = cdist(bad_mat, mat_cord, 'cosine')

In [ ]:
mm = MarkedMatrix((('real', good2cord), ('fake', bad2cord)))

In [ ]:
model = LogisticRegression(max_iter=1000, verbose=1, n_jobs=16, solver='lbfgs')

In [ ]:
mm.single_split_classify(model)

In [ ]:
def get_results(num, k=5):
    res = model.predict(bad2cord[num, :].reshape(1, -1))
    print(res)
    weights = model.coef_ * bad2cord[num, :].T
#     top_k = np.argsort(weights)[0, :k]
    top_k = np.argsort(weights)[0, :k]
    print('Clean Text: ', fake_df.iloc[num].text)
    for el in top_k:
        print('-'*25)
        print(f'Weight: {weights[0, el]}')
        print(f'Related abstract ({el}): {cord_df.iloc[el].abstract}')
        print(f'Related title ({el}): {cord_df.iloc[el].title}')

In [ ]:
get_results(930, k=10)

# 